<a href="https://colab.research.google.com/github/TimJackman/CS599HW/blob/main/Privacy_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import math
import pandas as pd

rng = np.random.default_rng(seed=42)

In [ ]:
def sign(z):
  return np.sign(z)

#Basic Score (too slow :()
def score(y,X):
  score = 0
  for xi in X:
    score += sign(y - xi)
  return -1 * abs(score)

def fasterScore(y,sortedX):
  greaterThanY = len(sortedX) - np.searchsorted(sortedX, y, side='right')
  lessThanY = np.searchsorted(sortedX, y, side='left')
  return -1 * abs(greaterThanY - lessThanY)

In [ ]:
def roundData(X,R):
  roundedX = []
  for xi in X:
    roundedxi = round(xi)
    if roundedxi < 1:
      roundedxi = 1
    elif roundedxi > R:
      roundedxi = R
    roundedX.append(roundedxi)
  
  return np.array(roundedX)


def computeScores(sortedX, R):
  scores = []
  for i in range(R):
    scores.append(fasterScore(i+1,X))
  return scores

def computeScoresSlow(X,R):
  scores = []
  for i in range(R):
    scores.append(score(i+1,X))
  return scores

def noisyMedian(scores,R,epsilon):
  Z = rng.exponential(2/epsilon,R)
  return np.argmax(scores + Z) + 1

In [ ]:
N = (50,100,500,2000,10000)
eps = 0.1

In [ ]:
data = pd.DataFrame(columns=['Distribution', 'n', 'R','Data Set Number', 'Run Number','Absolute Value of Score'])

In [ ]:
# Normal Distribution
RNormal = [2**7,2**10,2**16]
for R in RNormal:
  for n in N:
    for dataSet in range(50):
      X = np.sort(roundData(rng.normal(R/4, R/math.sqrt(10), n),R))
      scores = computeScores(X, R)
      for run in range(10):
        noisy_median = noisyMedian(scores, R, eps)
        noisy_median_score = fasterScore(noisy_median, X)
        
        result = ['Normal', n, R, dataSet, run, abs(noisy_median_score)]
        data.loc[len(data)] = result

In [ ]:
# Poisson Distribution
RPoisson = [2**7,2**10,2**16]
for R in RPoisson:
  for n in N:
    for dataSet in range(50):
      X = np.sort(roundData(rng.poisson(50, n),R))
      scores = computeScores(X, R)
      for run in range(10):
        noisy_median = noisyMedian(scores, R, eps)
        noisy_median_score = fasterScore(noisy_median, X)
        
        result = ['Poisson', n, R, dataSet, run, abs(noisy_median_score)]
        data.loc[len(data)] = result

In [ ]:
# Bimodal Distribution
kBimodal = [10,100,200]
R = 2**10
for k in kBimodal:
  for n in N:
    for dataSet in range(50):
      choices = [R/2 + k, R/2 - k]
      X = np.sort(rng.choice(choices, n))
      scores = computeScores(X, R)
      for run in range(10):
        noisy_median = noisyMedian(scores, R, eps)
        noisy_median_score = fasterScore(noisy_median, X)
        
        result = ['Bimodal', n, k, dataSet, run, abs(noisy_median_score)]
        data.loc[len(data)] = result

In [ ]:
from google.colab import files

data.to_csv('output.csv', encoding = 'utf-8-sig')
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>